In [ ]:
#Импортируем все необходимые библиотеки
import os
from PIL import Image, ImageDraw

In [ ]:
#Пункт 1. Обработка типовок
#получаем список всех изображений в папке
def get_files(save_w, way, tipovki_list):
    os.chdir(way)
    folders = os.listdir() #Получаем содержимое папки
    for i in range (0, len(folders)):
        if str(folders[i]).count('_') != True: #Если в названии папки нет знака "_"
            os.chdir(way + str(folders[i]) + '/') #переходим в эту папку
            tipovki_list.append(folders[i]) #в список tipovki_list добавляем название папки
            tipovki_list.append(os.listdir()) #в список tipovki_list добавляем список с именами файлов в папке
            #В папке, для обработанных изображений создадим (если их там нет) папки с названием ЖК и знаком "_" в конце
            os.chdir(save_w)
            try:
                os.mkdir(folders[i] + '_')
            except:
                pass
    return tipovki_list

#Обрабатываем изображение
def ghosts(direct, direct_save, name):
    os.chdir(direct)
    im = Image.open(name) #загружаем изображение
    im = im.convert("RGBA") #конвертируем изображение в RGB с альфа-каналом 
    width = im.size[0] #Определяем ширину изображения 
    height = im.size[1] #Определяем высоту изображения
    pix = im.load() #Выгружаем значения пикселей изображения

    #делаем границы прозрачными
    opacity_level = 0 # Задаем уровень прозрачности границ (вводится значения от 0 (абсолютно прозрачный) до 255 (абсолютно не прозрачный))
    center = (0, 2)
    yellow = (255, 255, 0, 0)
    ImageDraw.floodfill(im, center, yellow, thresh = 10) #заливаем изображение  желтым цветом с альфа каналом == 0, начиная со 2 пикселя 
    
    
    #обрезаем изображение
    left, up = 0, 0
    min_j, max_j = 0, height
    right, down = 0, 0
    min_i = 0 
    for i in range (width):
        for j in range (height):
            #print (pix[i, j][3])
            if pix[i, j][3] > 100 and left == 0: #ищем левую границу
                left = i 
                max_j =  j
            elif pix[i, j][3] > 100 and left != 0: 
                if j < max_j: #ищем верхнюю границу
                    up = j
                    max_j = j
                elif j > min_j: #ищем нижнюю границу
                    down =  j
                    min_j = j 
                elif i > min_i: #ищем правую границу
                    right = i
                    min_i = i
    area = (left, up, right, down)
    im = im.crop(area)
    
    #меняем размер изображения до 170*170 pix
    im2 = Image.new('RGBA', (170, 170), color=(255, 255, 255, 0))
    width2, height2 = im2.size[0], im2.size[1]
    canvas = 170 #Размер нового полотна (изображение, все пиксели белого цвета) в пикселях
    width_changed = im.size[0] #Определяем ширину изображения 
    height_changed = im.size[1] #Определяем высоту изображения
    #Изменяем размер исходного изображения зависимости от размеров сторон исходного изображения
    #(изменённое изображение вставляем в новое полотно)
    if width_changed > canvas and height_changed > canvas:
        im = im.resize((canvas, canvas), Image.BILINEAR)
        im2.paste(im)
    elif width_changed > canvas and height_changed < canvas:
        im = im.resize((canvas, canvas), Image.BILINEAR)
        im2.paste(im)
    elif width_changed > canvas and height_changed == canvas:
        im = im.resize((canvas, height_changed), Image.BILINEAR)
        im2.paste(im, box=(0, height2 - height_changed))
    elif width_changed < canvas and height_changed > canvas:
        im = im.resize((canvas, canvas), Image.BILINEAR)
        im2.paste(im)
    elif width_changed < canvas and height_changed < canvas:
        im = im.resize((canvas, canvas), Image.BILINEAR)
        im2.paste(im)
    elif width_changed < canvas and height_changed == canvas:
        im = im.resize((canvas, height_changed), Image.BILINEAR)
        im2.paste(im, box=(0, height2 - height_changed))
    elif width_changed == canvas and height_changed > canvas:
        im = im.resize((width_changed, canvas), Image.BILINEAR)
        im2.paste(im, box=(width2 - width_changed, 0))
    elif width_changed == canvas and height_changed < canvas:
        im = im.resize((width_changed, canvas), Image.BILINEAR)
        im2.paste(im, box=(width2 - width_changed, 0))
    elif width_changed == canvas and height_changed == canvas:
        im2.paste(im, box=(width2 - width_changed, height2 - height_changed))

    #Сохраняем изображение в формате .PNG 
    os.chdir(direct_save)
    im2.save(name[ : -4] + '_edit.png', "PNG")

#Объявляем функцию обработки типовок
def tipovki():
    img_list, tipovki_list = [], []
    way = '/content/drive/My Drive/Colab Notebooks/examples/tipovki_to_update/' #Здесь нужно указать путь к папкам, содержащим папки с исходными изображениями типовок
    save_w = '/content/drive/My Drive/Colab Notebooks/examples/tipovki/' #путь к папке, в которую нужно сохранить папки, содержащие обработанные изображения типовок
    get_files(save_w, way, tipovki_list) #выполняем функцию получения списка файлов
    for i in range (0, len(tipovki_list), 2):
        for j in range (0, len(tipovki_list[i + 1])):
            name = tipovki_list[i + 1][j] #name -строка состоящая из названия файла
            direct = way + tipovki_list[i] + '/' #путь к папке с исходными изображениями
            direct_save = save_w + tipovki_list[i] + '_' + '/' #путь к папке для сохранения обработанных изображений
            ghosts(direct, direct_save, name)
#Запускаем выполнение обработки типовок только если типовки есть в папке tipovki_to_update

os.chdir('/content/drive/My Drive/Colab Notebooks/examples/tipovki_to_update/')
new_tipovki = os.listdir()
if new_tipovki != []:
    tipovki() #запуск обработки типовок